In [ ]:
import numpy as np
import pandas as pd
from skimpy.analysis.oracle import *
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append("../../../../NRAplus/NRAplus") # Adds higher directory to python modules path.
sys.path.append('../')
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from tqdm.auto import tqdm
import plotly.graph_objects as go
import kaleido

from utils.nra_save_custom_json import load_json_nra_model
from pytfa.io.json import load_json_model

import configparser
import os

config = configparser.ConfigParser()
config.read(os.path.abspath('../src/config.ini'))
base_dir = config['paths']['base_dir']
path_to_tfa_samples_MUT = os.path.join(base_dir, config['paths']['path_to_samples_MUT'])
path_to_tfa_samples_WT = os.path.join(base_dir, config['paths']['path_to_samples_WT'])
path_to_tmodel_MUT = os.path.join(base_dir, config['paths']['path_to_tmodel_MUT'])
path_to_tmodel_WT = os.path.join(base_dir, config['paths']['path_to_tmodel_WT'])
path_to_nra_model = os.path.join(base_dir, config['paths']['path_to_nra_model'])
path_to_solutions = os.path.join(base_dir, config['paths']['path_to_solutions'])
path_to_essential_enzymes = os.path.join(base_dir, config['paths']['path_to_essential_enzymes'])
path_to_tflink_database = os.path.join(base_dir, config['paths']['path_to_tflink_database'])
path_to_recon_model = os.path.join(base_dir, config['paths']['path_to_recon_model'])
path_to_gene_to_uniprot_mapping = os.path.join(base_dir, config['paths']['path_to_gene_to_uniprot_mapping'])
path_to_essential_enzymes_to_TFs_mapping = os.path.join(base_dir, config['paths']['path_to_essential_enzymes_to_TFs_mapping'])  

In [ ]:
nra_model = load_json_nra_model(path_to_nra_model)

# Figure S9

In [ ]:
# Collect all the objective function values
obj_values = pd.DataFrame(columns=['enzymes_perturbed', 'obj_value'])
for max_enz_mod in [1, 2, 5, 10, 50, 100, 150, 200, 250, 500, 877]:
    # Load the solution
    sol = pd.read_csv(path_to_solutions.format(max_enz_mod), index_col=0)
    # Get the values of the auxilariy variables 
    aux_vars = [v for v in sol.index if v.startswith('AUXV_')]
    # Get the value of the objective function by adding the values of the auxiliary variables
    obj_value = sol.loc[aux_vars].sum().values[0]
    obj_values.loc[max_enz_mod] = obj_value

    # Find how many enzymes were perturbed
    epsilon = 1e-9
    active_enzyme_regulations = [v for v in sol.index if v.startswith('EU_') or v.startswith('ED_')]
    active_enzyme_regulations = [v for v in active_enzyme_regulations
                                    if np.abs(sol.loc[v].values[0]) > epsilon]
    print('Number of enzymes perturbed:', len(active_enzyme_regulations))
    print('Objective value:', obj_value, '\n')

    obj_values.loc[max_enz_mod] = [len(active_enzyme_regulations), obj_value]

original_value = 371.2092331511442

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))

# Plot the main curve
plt.plot(
    obj_values.enzymes_perturbed,
    obj_values.obj_value,
    marker='o',
    color='steelblue',
    linewidth=2,
    markersize=6,
    label='Difference after enzyme changes'
)

# Add the reference (original value)
plt.axhline(
    original_value,
    color='firebrick',
    linestyle='--',
    linewidth=2,
    label='Original Difference (no changes)'
)

# Labels for a biology audience
plt.xlabel('Number of enzyme activity changes', fontsize=16)
plt.ylabel('Difference in metabolic activity', fontsize=16)

# Axis styling
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Legend
plt.legend(fontsize=14, frameon=False, loc='center right')

# Clean aesthetics
plt.tight_layout()
plt.show()


# Figure 5A

In [ ]:
# Load the solution with 5 enzyme modifications
sol =  pd.read_csv(path_to_solutions.format(250), index_col=0)

# Identify the AUXV variables
aux_vars = [var for var in sol.index if var.startswith('AUXV_')]
modifications = [0, 100, 250, 877]
auxv_df = pd.DataFrame(index=aux_vars, columns=modifications)
for max_enz_mod in modifications:
    sol =  pd.read_csv(path_to_solutions.format(max_enz_mod), index_col=0)

    # Add to auxv_df DataFrame
    auxv_df[max_enz_mod] = sol.loc[aux_vars]


In [ ]:
# Sort auxvars per subsystem and find total subsystem deviations per solution
core_ss = ['Glycolysis/gluconeogenesis', 'Citric acid cycle', 'Pentose phosphate pathway', 'Glutamate metabolism',
'ROS detoxification', 'Glycine, serine, alanine, and threonine metabolism', 'Urea cycle', 'Arginine and proline metabolism',
'Purine synthesis', 'Pyrimidine synthesis', 'ETC_Rxns', 'NAD metabolism']
subs_deviations = pd.DataFrame(index=core_ss, columns=modifications).fillna(0)


for var in aux_vars:
    rxn = var.split('AUXV_',1)[1]
    rxn = nra_model.reactions.get_by_id(rxn)
    if rxn.subsystem in core_ss:
        for max_enz_mod in modifications:
            subs_deviations.loc[rxn.subsystem, max_enz_mod] += auxv_df.loc[var, max_enz_mod]

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Choose a nice color palette (e.g., Set2)
palette = plt.get_cmap('Set2').colors

# Filter to only the desired fold-changes
keep_mods = [0, 100, 250, 877]
data = subs_deviations[keep_mods]

# Rename index entries
data = data.rename(index={
    'ETC_Rxns': 'Oxidative phosphorylation',
    'Glycine, serine, alanine, and threonine metabolism': 'Gly/Ser/Ala/Thr metabolism',
    'Arginine and proline metabolism': 'Arg/Pro metabolism'
})

# Reorder the index to match the desired order
order = [
    'Gly/Ser/Ala/Thr metabolism', 'Arg/Pro metabolism', 'Glutamate metabolism',
    'Glycolysis/gluconeogenesis', 'Pentose phosphate pathway', 'Citric acid cycle',
    'Urea cycle', 'Pyrimidine synthesis', 'Purine synthesis',
    'NAD metabolism', 'ROS detoxification', 'Oxidative phosphorylation'
]
data = data.reindex(order)

# Create the bar plot
fig, ax = plt.subplots(figsize=(14, 6))

n_cats = len(data.index)
n_mods = len(data.columns)
bar_width = 0.8 / n_mods
x = np.arange(n_cats)

for i, mod in enumerate(data.columns):
    ax.bar(
        x + i * bar_width,
        data[mod].values,
        width=bar_width,
        label=f'{mod}',
        color=palette[i % len(palette)],
        edgecolor='black',
        linewidth=1
    )

# Ticks & labels
ax.set_xticks(x + bar_width * (n_mods - 1) / 2)
ax.set_xticklabels(data.index, rotation=45, ha='right', fontsize=12)
ax.tick_params(axis='y', labelsize=12)

ax.set_xlabel('Metabolic Pathway', fontsize=16)
ax.set_ylabel('Total pathway deviation', fontsize=16)

# Remove upper and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Place legend inside and remove its frame
legend = ax.legend(
    title='Enzyme fold changes applied',
    loc='upper right',
    frameon=False,
    fontsize=12,
    title_fontsize=16,
)

# Left-align the legend title and content box
legend.get_title().set_ha('left')
legend._legend_box.align = "left"

plt.tight_layout()

plt.rcParams['pdf.fonttype'] = 42  # For PDF
plt.rcParams['svg.fonttype'] = 'none'  # For SVG

plt.show()


# Figure S10

In [ ]:
tmodel_MUT = load_json_model(path_to_tmodel_MUT)
tmodel_WT = load_json_model(path_to_tmodel_WT)

In [ ]:
# Find how many reactions have opposite directionality in the two models.
# We will not include them in the objective function.
diff_dir_rxns = []
mut_rxns_ids = [rxn.id for rxn  in tmodel_MUT.reactions]
for rxn in tmodel_WT.reactions:
    if rxn.id in mut_rxns_ids:
        rxn_mut = tmodel_MUT.reactions.get_by_id(rxn.id)
        if rxn_mut.lower_bound*rxn.lower_bound < 0:
            diff_dir_rxns.append(rxn.id)

print('Number of reactions with opposite directionality in the two models:', len(diff_dir_rxns))

# Load the WT steady state samples and use PCA to determine which one is closest to the mean
samples_WT = pd.read_csv(path_to_tfa_samples_WT, index_col=0)

# Clean the samples to only include active fluxes and metabolite concentrations
active_fluxes = []
for r in tmodel_WT.reactions:
    if r.lower_bound > 0:
        active_fluxes.append(r.id)
    else:
        active_fluxes.append(r.reverse_id)

active_vars = [i for i in tmodel_WT.variables.keys() if i.startswith('LC_')]

samples_WT_clean = pd.DataFrame()
for name, vals in samples_WT.iteritems():
    if name in active_fluxes or name in active_vars:
        samples_WT_clean[name] = vals
        
samples_WT_clean = samples_WT_clean.loc[0:599, :]

# Select the reference sample for WT
reference_sample_index = 66
reference_sample_WT = samples_WT_clean.loc[reference_sample_index,:]

# Load the MUT steady state samples
tfa_ix = 17
tfa_sample = pd.read_csv(path_to_tfa_samples_MUT, header=0, index_col=0).iloc[tfa_ix]

In [ ]:
# Find how many reactions have opposite directionality in the two models.
# We will not include them in the objective function.
diff_dir_rxns = []
mut_rxns_ids = [rxn.id for rxn  in tmodel_MUT.reactions]
for rxn in tmodel_WT.reactions:
    if rxn.id in mut_rxns_ids:
        rxn_mut = tmodel_MUT.reactions.get_by_id(rxn.id)
        if rxn_mut.lower_bound*rxn.lower_bound < 0:
            diff_dir_rxns.append(rxn.id)

In [ ]:
# Gather all the optimization results and find the essential enzymes (always active)
# nra_model.max_enzyme_modifications = 250
epsilon = 1e-9
sol = pd.read_csv(path_to_solutions.format(250), index_col=0)
enzyme_use_vars_250 = [i for i in sol.index if i.startswith('EUDUSE_') and np.isclose(sol.loc[i][0],0)]
enzymes_used_250 = [i.split('EUDUSE_')[1] for i in enzyme_use_vars_250]

# Find all the reactions in subsystems glycokysis, oxidative phosphorylation and Fatty acid synthesis
subsystems_of_interest = core_ss = ['Glycolysis/gluconeogenesis', 'Citric acid cycle', 'Pentose phosphate pathway', 'Glutamate metabolism',
'ROS detoxification', 'Glycine, serine, alanine, and threonine metabolism', 'Urea cycle', 'Arginine and proline metabolism',
'Purine synthesis', 'Pyrimidine synthesis', 'ETC_Rxns', 'NAD metabolism']

df_effect = pd.DataFrame(columns=['Reaction', 'Subsystem'])
for rxn in nra_model.reactions:
    if rxn.subsystem in subsystems_of_interest and rxn.id not in diff_dir_rxns:
        if 'AUXV_'+rxn.id not in sol.index:
            print('Reaction {} is not in the solution'.format(rxn.id))
            # continue
        df_effect = df_effect.append({'Reaction': rxn.id, 'Subsystem': rxn.subsystem}, ignore_index=True)

df_effect.set_index('Reaction', inplace=True)

In [ ]:
# Calculate the effect of each essential enzyme on the fluxes fo each subsystem
for enz in tqdm(enzymes_used_250):
    df_effect[enz] = 0.0
    for rxn in df_effect.index:
        # Find control coefficient of the enzyme on the reaction
        frb = nra_model.constraints['FRB_{}'.format(rxn)]
        fcc = - frb.get_linear_coefficients([nra_model.variables['EU_{}'.format(enz)]])[nra_model.variables['EU_{}'.format(enz)]]

        # Find fcc*(EU_enz - ED_enz)
        eu_enz = sol.loc['EU_{}'.format(enz)][0]
        ed_enz = sol.loc['ED_{}'.format(enz)][0]
        df_effect.loc[rxn, enz] = fcc * (eu_enz - ed_enz) 

# Have a column with the ratio that we remove from the objective function i.e. ln(vWT/vMUT)
df_effect_const = pd.DataFrame(index=df_effect.index, columns=['ln_vWT_vMUT'])
# Add a column with the subsystem of the reaction
df_effect_const['Subsystem'] = df_effect['Subsystem']
df_effect_const['ln_vWT_vMUT'] = 0.0
for rxn in df_effect_const.index:
    rxn_var = nra_model.reactions.get_by_id(rxn)
    try:
        vwt = reference_sample_WT[rxn]
    except:
        vwt = reference_sample_WT[rxn_var.reverse_id]
    vmut = np.abs(tfa_sample[rxn_var.id] - tfa_sample[rxn_var.reverse_id])
    df_effect_const.loc[rxn, 'ln_vWT_vMUT'] = np.log(vwt / vmut)


In [ ]:
# Group the enzymes by susbsystems too and make a grouped df
df_effect_sum = df_effect.groupby('Subsystem').sum()
df_effect_grouped = pd.DataFrame()
for enz in df_effect_sum.columns:
    sub = nra_model.reactions.get_by_id(enz).subsystem
    effect = df_effect_sum.loc[:, enz]
    if sub not in df_effect_grouped.columns:
        df_effect_grouped[sub] = effect
    else:
        df_effect_grouped[sub] += effect

# Make sure the columns have the same order as the index of the df_effect_grouped
df_effect_grouped = df_effect_grouped.reindex(sorted(df_effect_grouped.columns), axis=1)


In [ ]:
# Keep only the subsystems that are in core_ss
df_effect_grouped_core = df_effect_grouped.loc[:, df_effect_grouped.columns.isin(core_ss)]

# Make sure the index sort is the same as the columns
df_effect_grouped_core = df_effect_grouped_core.reindex(sorted(df_effect_grouped_core.index), axis=0)
df_effect_grouped_core = df_effect_grouped_core.reindex(sorted(df_effect_grouped_core.columns), axis=1)

# Replace some names
df_effect_grouped_core = df_effect_grouped_core.rename(index={
    'ETC_Rxns': 'Oxidative phosphorylation',
    'Glycine, serine, alanine, and threonine metabolism': 'Gly/Ser/Ala/Thr metabolism',
    'Arginine and proline metabolism': 'Arg/Pro metabolism'
})

df_effect_grouped_core = df_effect_grouped_core.rename(columns={
    'ETC_Rxns': 'Oxidative phosphorylation',
    'Glycine, serine, alanine, and threonine metabolism': 'Gly/Ser/Ala/Thr metabolism',
    'Arginine and proline metabolism': 'Arg/Pro metabolism'
})

# Sort the index and columns based on 
order = [
    'Gly/Ser/Ala/Thr metabolism', 'Arg/Pro metabolism', 'Glutamate metabolism',
    'Glycolysis/gluconeogenesis', 'Pentose phosphate pathway', 'Citric acid cycle',
    'Urea cycle', 'Pyrimidine synthesis', 'Purine synthesis',
    'NAD metabolism', 'ROS detoxification', 'Oxidative phosphorylation'
]

df_effect_grouped_core = df_effect_grouped_core.reindex(index=order, columns=order)

# Set color scale limits (change these values as needed)
color_limit = 50

fig, ax = plt.subplots(figsize=(12, 8))

# Clamp values to [-color_limit, color_limit] range for color mapping
df_effect_clamped = df_effect_grouped_core.clip(lower=-color_limit, upper=color_limit)

# Draw heatmap with clamped values
sns.heatmap(
    df_effect_clamped,
    annot=False,
    cmap='RdBu_r',
    center=0,
    vmin=-color_limit,
    vmax=color_limit,
    fmt='.2f',
    linewidths=0.5,
    square=True,
    cbar=False,
    ax=ax
)

# Create small horizontal colorbar (1/5th width, thicker height) at top-right
cax = inset_axes(
    ax,
    width="20%",         
    height="5%",        
    loc='upper right',
    bbox_to_anchor=(0, 0, 1, 1.1),
    bbox_transform=ax.transAxes,
    borderpad=0
)

# Colorbar content - use clamped range
vmin = -color_limit
vmax = color_limit
norm = plt.Normalize(vmin=vmin, vmax=vmax)
sm = plt.cm.ScalarMappable(cmap='RdBu_r', norm=norm)
cb = fig.colorbar(sm, cax=cax, orientation='horizontal')

# Set custom tick values and labels
tick_values = [-color_limit, 0, color_limit]
tick_labels = [f' < -{color_limit}', '0', f' > {color_limit}']
cb.set_ticks(tick_values)
cb.set_ticklabels(tick_labels)
cb.ax.tick_params(labelsize=8, direction='out')
cax.xaxis.set_ticks_position("top")
cax.xaxis.set_label_position("top")

# Axis labels and ticks
# ax.set_xlabel('', fontsize=8)
ax.set_ylabel('Affected Pathway', fontsize=15)
ax.set_xlabel('Enzymes grouped according to their pathway', fontsize=15)
ax.tick_params(axis='y', labelsize=12, rotation=0)

# Apply ha='right' for x-axis tick labels
for label in ax.get_xticklabels():
    label.set_ha('right')
    label.set_rotation(45)
    label.set_fontsize(12)

plt.tight_layout()

plt.show()

# Figure 5B, 5C

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 1) Publication-grade font & style
plt.rcParams.update({
    'font.family': 'sans-serif',
    'font.sans-serif': ['Arial'],
    'font.size': 11,
    'pdf.fonttype': 42,
    'svg.fonttype': 'none'
})
sns.reset_defaults()
sns.set_style('white')

# 2) Define custom colors for positive/negative bars
positive_color = '#228B22'  # ForestGreen
negative_color = '#B22222'  # Firebrick

# 3) Subsystems and enzymes
subsystems = ['Glycolysis/gluconeogenesis', 'Pyrimidine synthesis']
enzymes = df_effect_grouped_core.columns

# 4) Prepare data matrix
data = df_effect_grouped_core.loc[subsystems].values

# 5) Loop over each subsystem and make its own figure
for sub, vals in zip(subsystems, data):
    fig, ax = plt.subplots(figsize=(8, 6), sharey=False)  # half the width
    # assign colors
    colors = [positive_color if v >= 0 else negative_color for v in vals]
    y = np.arange(len(enzymes))

    # plot horizontal bars
    ax.barh(y, vals, height=0.6, color=colors, edgecolor='black', linewidth=1)

    # zero line
    ax.axvline(0, color='black', linewidth=0.8, linestyle='--')

    # labels & title
    ax.set_yticks(y)
    ax.set_yticklabels(enzymes, rotation=0, ha='right', fontsize=12)
    ax.tick_params(axis='x', labelsize=12)
    ax.set_title(sub, fontsize=18)
    ax.set_ylabel('Effects of enzymes grouped by metabolic pathway', fontsize=14, labelpad=10)

    # x-label only on the first plot
    if sub == subsystems[0]:
        ax.set_xlabel('Effect on pathway', fontsize=14, labelpad=10)
    else:
        # slightly extend lower limit on pyrimidine panel
        x_min, x_max = ax.get_xlim()
        ax.set_xlim(-2, x_max)
        ax.set_xlabel('Effect on pathway', fontsize=14, labelpad=10)

    sns.despine(ax=ax)
    plt.tight_layout()

    # save each separately
    plt.show()


# Figure 5D

In [ ]:
# Read the essential enzymes txt file
with open(path_to_essential_enzymes, 'r') as f:
    essential_enzymes_list = [fline.strip() for fline in f.readlines()]

# Load the reference solution
sol = pd.read_csv(path_to_solutions.format('250'), index_col=0)

In [ ]:
# Save in a dataframe the enzyme name if it is up or down regulated and the subsystem it belongs to
essential_enzymes = pd.DataFrame(columns=['Enzyme', 'Subsystem', 'Regulation'])
for enz in essential_enzymes_list:
    var_up = sol.loc['EUU_' + enz].values[0]
    var_down = sol.loc['EDU_' + enz].values[0]
    if np.isclose(var_up, 1.0):
        regulation = 'up'
    elif np.isclose(var_down, 1.0):
        regulation = 'down'
    else:
        raise ValueError(f"Enzyme {enz} is not up or down regulated")

    rxn = nra_model.reactions.get_by_id(enz)
    sub = rxn.subsystem
    essential_enzymes = essential_enzymes.append({'Enzyme': enz, 'Subsystem': sub, 'Regulation': regulation}, ignore_index=True)

# Set the index to the enzyme name
essential_enzymes.set_index('Enzyme', inplace=True)

# Print how many enzymes are up or down regulated 
print(f"Number of down regulated enzymes: {len(essential_enzymes[essential_enzymes['Regulation'] == 'down'])}")
print(f"Number of up regulated enzymes: {len(essential_enzymes[essential_enzymes['Regulation'] == 'up'])}")

In [ ]:
# Set the style for high-impact journals
plt.style.use('default')
sns.set_palette("Set2")

# Count number of up- and down-regulated enzymes per subsystem
df = essential_enzymes.copy()

# Rename the long subsystem
df['Subsystem'] = df['Subsystem'].replace(
    'Glycine, serine, alanine, and threonine metabolism',
    'Gly/Se/rAla/Theo metabolism'
)

# Lump subsystems with fewer than 8 enzymes into "Other"
subsystem_counts = df['Subsystem'].value_counts()
rare_subsystems = subsystem_counts[subsystem_counts < 8].index
df['Subsystem'] = df['Subsystem'].replace(rare_subsystems, 'Other')

# Group and count
count_df = df.groupby(['Subsystem', 'Regulation']).size().reset_index(name='Count')

# Ensure 'Other' is at the end
subsystem_order = [s for s in count_df['Subsystem'].unique() if s != 'Other']
subsystem_order.sort()
subsystem_order.append('Other')

colors = ['#2E86AB', '#A23B72']  # Blue and burgundy

# Calculate total essential enzymes per subsystem and sort
totals = [
    "Gly/Ser/Ala/Thr metabolism",
    "Arginine and proline metabolism",
    "Glutamate metabolism",
    "Glycolysis/gluconeogenesis",
    "Pentose phosphate pathway",
    "Citric acid cycle",
    "Urea cycle",
    "Pyrimidine synthesis",
    "NAD metabolism",
    "Other"
]


fig, ax = plt.subplots(figsize=(4, 6))  # taller than it is wide
sns.barplot(
    data=count_df,
    y='Subsystem',
    x='Count',
    hue='Regulation',
    order=totals,
    palette=colors,
    ax=ax,
    edgecolor='black',
    linewidth=1
)

# swap labels
ax.set_ylabel('Metabolic Pathway', fontsize=12)
ax.set_xlabel('Number of Essential Enzyme Changes', fontsize=12)

# keep pathway names horizontal
plt.yticks(rotation=0)
ax.tick_params(axis='y', which='both', length=0)  # ensure no little stubs


# move legend to the right, outside the axes
legend = ax.legend(
    loc='center',
    bbox_to_anchor=(1.0, 1.0),
    frameon=False
)

# rename legend entries
legend.texts[0].set_text('Downregulated')
legend.texts[1].set_text('Upregulated')

# remove top and right spines
sns.despine(ax=ax, top=True, right=True)

# give a bit of extra right margin so the legend isn’t cut off
plt.tight_layout(rect=[0, 0, 0.85, 1])

plt.show()

# Figure 5E

In [ ]:
# Load the TFLink database
tf_link = pd.read_csv(path_to_tflink_database, sep='\t')

from cobra.io.json import load_json_model # FBA type import
recon = load_json_model(path_to_recon_model)

# Load gene to UNIPROT mapping
gene_uniprot_mapping = pd.read_csv(path_to_gene_to_uniprot_mapping, sep='\t')

# The first column should be a string
gene_uniprot_mapping.iloc[:, 0] = gene_uniprot_mapping.iloc[:, 0].astype(str)

In [ ]:
brca1_total = tf_link[tf_link['Name.TF']=='BRCA1']

# Find which genes from recon3d are targeted by BRCA1
genes_identified = []
for ll in gene_uniprot_mapping.index:
    ncbi = gene_uniprot_mapping.loc[ll,'gene_number'].split('.')[0]
    if str(ncbi) in tf_link.loc[:,'NCBI.GeneID.Target'].values:
        row = tf_link[tf_link.loc[:,'NCBI.GeneID.Target'] == str(ncbi)]
        print(row)
        genes_identified.append(row)

# Make into a datframe
genes_identified = pd.concat(genes_identified, ignore_index=True)
genes_identified = genes_identified.drop_duplicates(ignore_index=True)

# Now we go from those TFs as targets to BRCA1 as a TF
tfs_identified = []
for ll in genes_identified.index:
    tf_name = genes_identified.loc[ll,'Name.TF']
    if tf_name in brca1_total.loc[:,'Name.TF'].values:
        row = brca1_total[brca1_total.loc[:,'Name.TF'] == tf_name]
        print(row)
        tfs_identified.append(row)

# Make into a datframe
tfs_identified = pd.concat(tfs_identified, ignore_index=True)
tfs_identified = tfs_identified.drop_duplicates(ignore_index=True)

# Go back to genes through the TFS identified in tfs_identified
brca1_specific_genes = []
for ll in tfs_identified.index:
    tf_name = tfs_identified.loc[ll,'Name.Target']
    if tf_name in genes_identified.loc[:,'Name.TF'].values:
        row = genes_identified[genes_identified.loc[:,'Name.TF'] == tf_name]
        print(row)
        brca1_specific_genes.append(row)

# Make into a datframe
brca1_specific_genes = pd.concat(brca1_specific_genes, ignore_index=True)
brca1_specific_genes = brca1_specific_genes.drop_duplicates(ignore_index=True)

In [ ]:
enz = []
for i in recon.genes:
    name = i.id.split('_')[0]
    if name in brca1_specific_genes.loc[:,'NCBI.GeneID.Target'].values:
        print(name)

        for rxn in i.reactions:
            if rxn.id not in enz:
                enz.append(rxn.id)
                print(rxn.id)
print(f'Total number of reactions associated with BRCA1: {len(enz)}')

# Do the same for the small model
enz_small = []
for i in nra_model.genes:
    name = i.id.split('.',1)[0]
    if name in brca1_specific_genes.loc[:,'NCBI.GeneID.Target'].values:
        print(name)

        for rxn in i.reactions:
            if rxn.id not in enz_small:
                enz_small.append(rxn.id)
                print(rxn.id)
print(f'Total number of reactions associated with BRCA1 in the small model: {len(enz_small)}')

In [ ]:
# For each essential enzyme find the genes ascosiated with it
essential_enzymes['genes'] = pd.Series(dtype=object) # Initialize with object dtype
essential_enzymes['uniprot_genes'] = pd.Series(dtype=object) # Initialize with object dtype
essential_enzymes['hugo_genes'] = pd.Series(dtype=object) # Initialize with object dtype

# Iterate through each essential enzyme (reaction ID)
for enz in essential_enzymes.index:
    rxn = nra_model.reactions.get_by_id(enz)
    genes = [g.id for g in rxn.genes]

    if genes:
        # Join multiple gene IDs with ';'
        essential_enzymes.at[enz, 'genes'] = ';'.join(genes)
        # Find the corresponding uniprot IDs
        uniprot_ids = []
        for gene in genes:
            if gene in gene_uniprot_mapping['gene_number'].values:
                uniprot_id = gene_uniprot_mapping.loc[gene_uniprot_mapping['gene_number'] == gene, 'uniprot_gname'].values[0]
                if uniprot_id not in uniprot_ids and not pd.isna(uniprot_id):
                    uniprot_ids.append(uniprot_id)
        # Join multiple uniprot IDs with ';'
        if uniprot_ids:
            essential_enzymes.at[enz, 'uniprot_genes'] = ';'.join(uniprot_ids)
        else:
            # Keep as NaN if no uniprot IDs are found
            essential_enzymes.at[enz, 'uniprot_genes'] = np.nan

        # Find the corresponding HUGO gene names
        hugo_genes = []
        for gene in genes:
            if gene in gene_uniprot_mapping['gene_number'].values:
                hugo_id = gene_uniprot_mapping.loc[gene_uniprot_mapping['gene_number'] == gene, 'symbol'].values[0]
                if hugo_id not in hugo_genes and not pd.isna(hugo_id):
                    hugo_genes.append(hugo_id)
        # Join multiple HUGO gene names with ';'
        if hugo_genes:
            essential_enzymes.at[enz, 'hugo_genes'] = ';'.join(hugo_genes)
        else:
            # Keep as NaN if no HUGO gene names are found
            essential_enzymes.at[enz, 'hugo_genes'] = np.nan
    else:
        # Keep as NaN if no genes are associated
        essential_enzymes.at[enz, 'genes'] = np.nan

In [ ]:
# Check if TFLink returns more TF hits for the essential enzymes
essential_enzymes['TF_link'] = pd.Series(dtype=object) # Initialize with object dtype

for enz in essential_enzymes.index:
    uniprot_ids = essential_enzymes.loc[enz, 'uniprot_genes']
    if not pd.isna(uniprot_ids):
        uniprot_ids_split = uniprot_ids.split(';')
        tf_hits = []
        for uniprot_id in uniprot_ids_split:
            tf_matches = tf_link.loc[tf_link['UniprotID.Target'] == uniprot_id, 'Name.TF']
            if not tf_matches.empty:
                tf_hits.extend(tf_matches.unique())
        if tf_hits:
            essential_enzymes.at[enz, 'TF_link'] = ';'.join(tf_hits)
        else:
            essential_enzymes.at[enz, 'TF_link'] = np.nan

In [ ]:
# Find how many unique genes are there
gene_list = []
for i in essential_enzymes['genes'].dropna():
    genes = i.split(';')
    for gene in genes:
        gene_list.append(gene)
print(f"{essential_enzymes.genes.notna().sum()}/{len(essential_enzymes)} essential enzymes have genes associated with them")
print(f"Number of genes: {len(gene_list)}")
print(f"Number of unique genes: {len(set(gene_list))}\n")

# Find how many unique TFs are there for TF_link
tfs_list_tf_link = []
for i in essential_enzymes['TF_link'].dropna():
    tfs = i.split(';')
    for tf in tfs:
        tfs_list_tf_link.append(tf)
print(f"{essential_enzymes.TF_link.notna().sum()}/{len(essential_enzymes)} essential enzymes have TF_link TFs associated with them")
print(f"Number of TFs: {len(tfs_list_tf_link)}")
print(f"Number of unique TFs: {len(set(tfs_list_tf_link))}\n")

# Find the unique genes that are associated with the TF_link TFs
genes_tfs_tf_link = []
for i in essential_enzymes[essential_enzymes.TF_link.notna()].genes:
    genes = i.split(';')
    for gene in genes:
        genes_tfs_tf_link.append(gene)
print(f"Number of genes associated with TF_link TFs: {len(genes_tfs_tf_link)}")
print(f"Number of unique genes associated with TF_link TFs: {len(set(genes_tfs_tf_link))}\n")
print('------------------------------------------------------')

print(f'In total {essential_enzymes.TF_link.notna().sum()}/{len(essential_enzymes)} essential enzymes are connected to {len(set(genes_tfs_tf_link))} unique genes and are regulated by {len(set(tfs_list_tf_link))} unique TFs (TF_link)')

In [ ]:
# Load the CSV file
essential_enzymes.to_csv(path_to_essential_enzymes_to_TFs_mapping)
df = pd.read_csv(path_to_essential_enzymes_to_TFs_mapping)

# Change the Subsystems ETC_Rxns to Electron transport chain
df['Subsystem'] = df['Subsystem'].replace('ETC_Rxns', 'Electron transport chain')
# Drop rows missing TF or gene
df_filtered = df.dropna(subset=['TF_link', 'hugo_genes'])

# Split semicolon-separated values
df_filtered['hugo_genes'] = df_filtered['hugo_genes'].astype(str).str.split(';')
df_filtered['TF_link'] = df_filtered['TF_link'].astype(str).str.split(';')

# Explode all combinations
df_exploded = df_filtered.explode('hugo_genes').explode('TF_link')
df_exploded['hugo_genes'] = df_exploded['hugo_genes'].str.strip()
df_exploded['TF_link'] = df_exploded['TF_link'].str.strip()

# !!! IMPORTANT: Apply NADH2_u10mi merge before filtering to top TFs if we want this aggregated enzyme
# to be considered in the overall flow calculation for top TFs or relevant enzymes.
# If the merge is done later, the 'Enzyme' column for NADH2_u10mi might still be the original for earlier steps.
# Let's keep it here as it was.
df_exploded['hugo_genes'] = df_exploded.apply(
    lambda row: 'NADH2_u10mi genes' if row['Enzyme'] == 'NADH2_u10mi' else row['hugo_genes'],
    axis=1
)

# Filter to top 5 most common TFs based on all original connections
top_5_tfs_all_connections = df_exploded['TF_link'].value_counts().nlargest(5).index.tolist()
top_5_tfs_all_connections = ['SP1', 'MYC', 'RELA', 'NFKB1', 'AR', 'HIF1A', 'STAT1', 'EGR1', 'TP53']  # Use the same top TFs as before
df_top = df_exploded[df_exploded['TF_link'].isin(top_5_tfs_all_connections)].copy() # Use .copy() to avoid SettingWithCopyWarning

# Filter relevant columns and remove rows without Subsystem info
df_top.dropna(subset=['Subsystem'], inplace=True)


# --- REVISED LOGIC FOR ORDERING NODES BY FLOW ---

# 1. Prepare link dataframes with calculated 'value' for sorting
#    These are the actual links that will be drawn, so their 'value' is what we sort by.

# TF -> Enzyme links
tf_enzyme_flows = df_top.groupby(['TF_link', 'Enzyme']).size().reset_index(name='value')

# Enzyme -> Subsystem links
enzyme_subsystem_flows = df_top.groupby(['Enzyme', 'Subsystem']).size().reset_index(name='value')

# 2. Calculate total 'flow' for each node type (what's coming into it or leaving it)

# TFs: Sort by their total outgoing flow to enzymes (which is their primary contribution)
tf_outgoing_flow = tf_enzyme_flows.groupby('TF_link')['value'].sum().sort_values(ascending=False)
sorted_tfs = tf_outgoing_flow.index.tolist()

# Enzymes: Sort by their total incoming flow from TFs.
# This aligns with "inputs in each layer ... biggest inflow to smallest".
# NADH2_u10mi is already handled in df_exploded before this step.
enzyme_inflow_sum = tf_enzyme_flows.groupby('Enzyme')['value'].sum().sort_values(ascending=False)

# Ensure only enzymes present in the filtered df_top are considered
relevant_enzymes_in_df_top = df_top['Enzyme'].unique().tolist()
# Filter enzyme_inflow_sum to include only relevant enzymes
sorted_enzymes = [e for e in enzyme_inflow_sum.index.tolist() if e in relevant_enzymes_in_df_top]


# Subsystems: Sort by their total incoming flow from Enzymes.
subsystem_inflow_sum = enzyme_subsystem_flows.groupby('Subsystem')['value'].sum().sort_values(ascending=False)

# Ensure only subsystems present in the filtered df_top are considered
relevant_subsystems_in_df_top = df_top['Subsystem'].unique().tolist()
# Filter subsystem_inflow_sum to include only relevant subsystems
sorted_subsystems = [s for s in subsystem_inflow_sum.index.tolist() if s in relevant_subsystems_in_df_top]


# 3. Combine all nodes in the desired order for Sankey
# This order is CRITICAL for Plotly's rendering.
nodes = sorted_tfs + sorted_enzymes + sorted_subsystems
node_indices = {name: i for i, name in enumerate(nodes)}

# --- END REVISED LOGIC ---

# Build Sankey links using the prepared flow dataframes
source, target, value = [], [], []

# TF → Enzyme links
for _, row in tf_enzyme_flows.iterrows():
    # Only add links if both source and target nodes exist in our final `nodes` list
    if row['TF_link'] in node_indices and row['Enzyme'] in node_indices:
        source.append(node_indices[row['TF_link']])
        target.append(node_indices[row['Enzyme']])
        value.append(row['value'])

# Enzyme → Subsystem links
for _, row in enzyme_subsystem_flows.iterrows():
    # Only add links if both source and target nodes exist in our final `nodes` list
    if row['Enzyme'] in node_indices and row['Subsystem'] in node_indices:
        source.append(node_indices[row['Enzyme']])
        target.append(node_indices[row['Subsystem']])
        value.append(row['value'])


# Node colors (using the new sorted lists for checks)
node_colors = []
for node in nodes:
    if node in sorted_tfs:
        node_colors.append('rgba(255, 127, 14, 0.8)')  # TF = orange
    elif node in sorted_enzymes:
        node_colors.append('rgba(44, 160, 44, 0.8)') # Enzyme = green
    else: # Subsystem
        node_colors.append('rgba(31, 119, 180, 0.8)') # Subsystem = blue

# Link colors by source type (using the new sorted lists for checks)
link_colors = []
for s_idx in source: # s_idx is the index in the 'nodes' list
    if nodes[s_idx] in sorted_tfs:
        link_colors.append('rgba(255, 127, 14, 0.3)')  # TF→Enzyme (orange-ish)
    elif nodes[s_idx] in sorted_enzymes:
        link_colors.append('rgba(44, 160, 44, 0.3)')   # Enzyme→Subsystem (green-ish)

# Create Sankey plot
fig = go.Figure(data=[go.Sankey(
    textfont=dict(size=21, family='Arial, sans-serif'),  # <-- add this
    node=dict(
        pad=20,
        thickness=25,
        line=dict(color="black", width=0.8),
        label=nodes, # This is the crucial part for ordering
        color=node_colors,
        hovertemplate='<b>%{label}</b><br>Total flow: %{value}<extra></extra>'
    ),
    link=dict(
        source=source,
        target=target,
        value=value,
        color=link_colors,
        hovertemplate='<b>%{source.label}</b> → <b>%{target.label}</b><br>Flow: %{value}<extra></extra>'
    )
)])

fig.update_layout(
    title={
        'text': "",
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 16, 'family': 'Arial, sans-serif'}
    },
    font=dict(size=16, family='Arial, sans-serif'),
    plot_bgcolor='rgba(0,0,0,0)',    # transparent plotting area
    paper_bgcolor='rgba(0,0,0,0)',   # transparent paper/background
    width=2400,
    height=1200,
    margin=dict(l=50, r=50, t=80, b=50)
)

# TFs title in orange
fig.add_annotation(dict(
    x=0,
    y=0.95,
    xref='paper',
    yref='paper',
    text='TFs',
    showarrow=False,
    font=dict(size=28, family='Arial, sans-serif',
              color='rgba(255,127,14,1)')  # orange
))

# Enzymes title in green
fig.add_annotation(dict(
    x=0.5,
    y=1.03,
    xref='paper',
    yref='paper',
    text='Enzymes',
    showarrow=False,
    font=dict(size=28, family='Arial, sans-serif',
              color='rgba(44,160,44,1)')   # green
))

# Pathways title in blue
fig.add_annotation(dict(
    x=1.015,
    y=1.03,
    xref='paper',
    yref='paper',
    text='Pathways',
    showarrow=False,
    font=dict(size=28, family='Arial, sans-serif',
              color='rgba(31,119,180,1)')  # blue
))

# # Save the figure to pdf
# fig.write_image("sankey_plot.png", format="png", scale=3)

# Save the figure as pdf


fig.show()